# TMS_GeoProc HS 2018

# Rasterdaten im Quadtree




In [ ]:
import PIL    # Installation mit: pip install pillow
from IPython.core.display import HTML

## Umgang mit Bilddateien im Jupyter Notebook

### Ein Bild (von URL) im Jupyter Notebook darstellen

In [ ]:
url = "https://www.google.ch/images/branding/googlelogo/2x/googlelogo_color_272x92dp.png"
HTML('<img src="' + url + '" width="20%"></img>')

### Bild manuell herunterladen und darstellen

Verwenden der urllib2: https://docs.python.org/3/howto/urllib2.html

In [ ]:
from urllib.request import urlopen

outputfile = "test.png"

response = urlopen(url) 
with open(outputfile, 'wb') as f:
    while True:
        chunk = response.read(16*1024)
        if not chunk:
            break
        f.write(chunk)
        
        
HTML('<img src="' + outputfile + '" width="20%"></img>')

nun haben wir ein file "test.png" erstellt.
Es ist auch möglich Bilder direkt im Speicher zu halten ohne jedesmal ein File zu erstellen.

Python hat dazu ein Basis-Modul "io", welches "BytesIO" zur Verfügung stellt.

In [ ]:
from urllib.request import urlopen
from io import BytesIO
import base64


response = urlopen(url) 
buffer = BytesIO()

while True:
    chunk = response.read(16*1024)
    if not chunk:
        break
    buffer.write(chunk)
        
        
img_str = str(base64.b64encode(buffer.getvalue()), encoding="ascii")
HTML('<img src="data:image/png;base64,' + img_str + '" width="20%"></img>')

Read kann bei kleineren Files auch direkt aufgerufen werden. Da wir im Speicher sowieso nur kleiner Files halten können, machen wir das doch so:

In [ ]:
from urllib.request import urlopen
from io import BytesIO
import base64

response = urlopen(url)
buffer = BytesIO(response.read())
    
img_str = str(base64.b64encode(buffer.getvalue()), encoding="ascii")
HTML('<img src="data:image/png;base64,' + img_str + '" width="20%"></img>')

### Bilder analysieren

Wir wollen natürlich die Bilder nicht einfache im Jupyter Notebook darstellen, sondern wir wollen damit arbeiten.

Python unterstützt direkt keine Bildformate, es gibt aber zahlreiche Bibliothken, welche das übernehmen. Ein sehr populäres modul ist "PIL" (pillow). Dieses kann mit:


    pip install pillow
    
    
installiert werden.

In [ ]:
from PIL import Image
from urllib.request import urlopen
from io import BytesIO
import numpy as np

response = urlopen(url)
buffer = BytesIO(response.read())

im = Image.open(buffer)
print(im.format, im.size, im.mode)
im.close()

In [ ]:
from PIL import Image
from urllib.request import urlopen
from io import BytesIO
import numpy as np

response = urlopen(url)
buffer = BytesIO(response.read())

im = Image.open(buffer)

resized = im.resize((54,18))

data_full = np.array(im)
data = np.array(resized)
im.close()

print(data.shape)   # row, columns, value (RGB, RGBA, ...)

for y in range(data.shape[0]):
    for x in range(data.shape[1]):
        r = data[y][x][0]
        g = data[y][x][1]
        b = data[y][x][2]
        
        if r!=0 and g!=0 and b!=0:
            print("#", end="")
        else:
            print(".", end="")
    print("")


In [ ]:
data

Die Bilddaten aus numpy können natürlich auch mit matplotlib dargestellt werden:

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt


plt.imshow(data)
plt.show()

Eventuell ist es sinnvolle eine andere Interpolation zu verwenden.

Akzeptierte Filter sind: ‘none’, ‘nearest’, ‘bilinear’, ‘bicubic’, ‘spline16’, ‘spline36’, ‘hanning’, ‘hamming’, ‘hermite’, ‘kaiser’, ‘quadric’, ‘catrom’, ‘gaussian’, ‘bessel’, ‘mitchell’, ‘sinc’, ‘lanczos’

In [ ]:
plt.imshow(data, interpolation='spline36')
plt.show()

Wir haben übrigens immer noch die volle Auflösung des Bildes im numpy_array "data_full" (siehe oben)

In [ ]:
plt.imshow(data_full, interpolation='nearest')
plt.show()

## Rechnen im Quadtree

### Quadkey zu normalisierten Koordinaten

In [ ]:
def QuadKeyToNormalizedCoord(key):
    zoomlevels = len(key)
    x = 0
    y = 0
    scale = 1.0
    for i in range(0,zoomlevels):
        scale /= 2.0
        if key[i] == "0":
            y += scale
        elif key[i] == "1":
            x += scale
            y += scale
        elif key[i] == "2":
            pass
        elif key[i] == "3":
            x += scale

    return [ x, y , x + scale, y + scale]
    #return [ x*2.-1.,  (y)*2.-1, (x + scale)*2-1, (y + scale)*2.-1.]

In [ ]:
QuadKeyToNormalizedCoord("")

Da wir die beste floating-point Präzision wollen, macht es mehr Sinn im Bereich -1 bis 1 zu arbeiten.  
Es macht deshalb am meisten Sinn den Quadtree (Mercator Projektion, EPSG:3857) auf den Bereich (-1,-1) bis (1,1) zu normalisieren. Wobei (-1,-1) die Koordinate unten links und (1,1) oben rechts repräsentiert.

```
           (1,1)
      +-----+
      |     |
      |     |
      +-----+
  (-1,-1)
```

Siehe auch:
* http://spatialreference.org/ref/sr-org/6864/

In [ ]:
import pyproj

webmercator = pyproj.Proj(init='EPSG:3857')
wgs84 = pyproj.Proj(init='EPSG:4326')

print(pyproj.transform(wgs84, webmercator, -180, -85.05112877980659))
print(pyproj.transform(wgs84, webmercator, 180, 85.05112877980659))


In [ ]:
def QuadKeyToNormalizedMercatorCoord(key):
    zoomlevels = len(key)
    x = 0
    y = 0
    scale = 1.0
    for i in range(0,zoomlevels):
        scale /= 2.0
        if key[i] == "0":
            y += scale
        elif key[i] == "1":
            x += scale
            y += scale
        elif key[i] == "2":
            pass
        elif key[i] == "3":
            x += scale

    return [ x*2.-1.,  (y)*2.-1, (x + scale)*2-1, (y + scale)*2.-1.]

In [ ]:
QuadKeyToNormalizedMercatorCoord("012120012")

### Tile-Koordinaten zu Quadkey

In [ ]:
# bit-Shifting

print(1 << 2)
print(1 << 3)
print(1 << 4)
print(1 << 5)

In [ ]:
def Tilecoord2Quadkey(tx, ty, zoom):
    key = ""
    for i in range(zoom, 0, -1):
        digit = 0
        mask = 1 << (i - 1)
        if (tx & mask) != 0:
            digit += 1
        if (ty & mask) != 0:
            digit += 2
        key += str(digit)
    return key

In [ ]:
print(Tilecoord2Quadkey(0,0,1)) # "0"
print(Tilecoord2Quadkey(1,0,1)) # "1"
print(Tilecoord2Quadkey(0,1,1)) # "2"
print(Tilecoord2Quadkey(1,1,1)) # "3"

In [ ]:
# TMS kehrt y um (nur zur Info...)
def TMS2Quadkey(tx, ty, zoom):
    return Tilecoord2Quadkey(tx, (2**zoom-1)-ty, zoom)

## Mercator nach WGS84

    Snyder, John Parr. Map Projections - a Working Manual. Washington: U.S. G.P.O., 1987.

In [ ]:
import math

def NormalizedMercatorToWGS84(x, y):
    x = x * math.pi
    y = y * math.pi
   
    t = math.exp(-y);   
    lat = math.pi/2 - 2.0 * math.atan(t)
    lng = x / 1.0
    return [lng * 57.295779513082320876798154814105, lat * 57.295779513082320876798154814105]

In [ ]:
NormalizedMercatorToWGS84(-1,-1)

In [ ]:
print(NormalizedMercatorToWGS84(-1,-1))
print(NormalizedMercatorToWGS84(0,0))
print(NormalizedMercatorToWGS84(1,1))

## Quadkey zu WGS84

In [ ]:
import math

def QuadkeyToWGS84(quadcode):
    mercator = QuadKeyToNormalizedMercatorCoord(quadcode)
    t0 = NormalizedMercatorToWGS84(mercator[0],mercator[1])
    t1 = NormalizedMercatorToWGS84(mercator[2],mercator[3])
    return [t0[0], t0[1],  t1[0], t1[1]]

In [ ]:
print(QuadkeyToWGS84(""))
print(QuadkeyToWGS84("0"))
print(QuadkeyToWGS84("1"))
print(QuadkeyToWGS84("2"))
print(QuadkeyToWGS84("3"))

## WGS84 (Punkt) nach Tile

In [ ]:
def WGS84ToTile(lng, lat, zoom):
    # bl = NormalizedMercatorToWGS84(-1,-1)
    # tr = NormalizedMercatorToWGS84(1,1)
    
    MinLongitude = -180  # bl[0]
    MaxLongitude = 180   # tr[0]
    MinLatitude = -85.05112877980659 # bl[1]
    MaxLatitude = 85.05112877980659 # tr[1]

    mapSize = pow(2, zoom) * 256   # Tile Size 256x256

    if lng<MinLongitude:
        lng = MinLongitude
    elif lng>MaxLongitude:
        lng = MaxLongitude
        
    if lat<MinLatitude:
        lat = MinLatitude
    elif lat>MaxLatitude:
        lat = MaxLatitude

    p = [0,0,0]
    p[0] = int((lng + 180.0) / 360.0 * (2**zoom))
    p[1] = int((1.0 - math.log(math.tan(lat * math.pi / 180.0) 
                + 1.0 / math.cos(lat * math.pi / 180.0)) / math.pi) / 2.0 * (2**zoom))
    p[2] = zoom
    return p

In [ ]:
print(WGS84ToTile(8.529416,47.379133,5))
print(WGS84ToTile(8.529416,47.379133,12))
print(WGS84ToTile(8.529416,47.379133,18))

In [ ]:
def WGS84ToQuadkey(lng, lat, zoom):
    t = WGS84ToTile(lng, lat, zoom)
    return Tilecoord2Quadkey(t[0],t[1],zoom)

In [ ]:
print(WGS84ToQuadkey(8.529416,47.379133,5))
print(WGS84ToQuadkey(8.529416,47.379133,12))
print(WGS84ToQuadkey(8.529416,47.379133,18))

## Tile-Koordinate nach WGS84

In [ ]:
def TileToWGS84(x, y, zoom):
    key = Tilecoord2Quadkey(x,y,zoom)
    return QuadkeyToWGS84(key)

### Tile Server

Datensätze aus Amazon AWS (EPSG:3857)

* https://aws.amazon.com/de/public-datasets/terrain/
* https://mapzen.com/blog/elevation/


Tiles:

* https://s3.amazonaws.com/elevation-tiles-prod/terrarium/{z}/{x}/{y}.png
* https://s3.amazonaws.com/elevation-tiles-prod/normal/{z}/{x}/{y}.png
* https://s3.amazonaws.com/elevation-tiles-prod/geotiff/{z}/{x}/{y}.tif
* https://s3.amazonaws.com/elevation-tiles-prod/skadi/{N|S}{y}/{N|S}{y}{E|W}{x}.hgt.gz


Terrarium:
   
     elv = (red * 256 + green + blue / 256) - 32768
     
     
OpenStreetMap Tiles:

https://wiki.openstreetmap.org/wiki/Tile_servers

Beispiel: https://a.tile.openstreetmap.org/z/x/y.png 

### Matterhorn

In [ ]:
matterhorntile = WGS84ToTile(7.657476, 45.977871, 11)
matterhorntile

In [ ]:
WGS84ToQuadkey(7.657476, 45.977871, 11)

In [ ]:
TileToWGS84(1067, 728, 11)

In [ ]:
url = "https://s3.amazonaws.com/elevation-tiles-prod/normal/{z}/{x}/{y}.png"
urldata = "https://s3.amazonaws.com/elevation-tiles-prod/terrarium/{z}/{x}/{y}.png"
osm = "https://a.tile.openstreetmap.org/{z}/{x}/{y}.png"

matterhorn = url.format(x=matterhorntile[0],y=matterhorntile[1],z=matterhorntile[2])
matterhornosm = osm.format(x=matterhorntile[0],y=matterhorntile[1],z=matterhorntile[2])

print(matterhorn)
print(matterhornosm)

In [ ]:
from IPython.core.display import HTML

HTML('<img src="' + matterhorn + '"></img>')

In [ ]:
HTML('<img src="' + matterhornosm + '"></img>')

In [ ]:
BaselTile = WGS84ToTile(7.592427, 47.556467, 11)
BaselTile

In [ ]:
baselosm = osm.format(x=BaselTile[0],y=BaselTile[1],z=BaselTile[2])
HTML('<img src="' + baselosm + '"></img>')

In [ ]:
basel = url.format(x=BaselTile[0],y=BaselTile[1],z=BaselTile[2])
HTML('<img src="' + basel + '"></img>')

In [ ]:
basel = urldata.format(x=BaselTile[0],y=BaselTile[1],z=BaselTile[2])
HTML('<img src="' + basel + '"></img>')

In [ ]:
matterhorn = urldata.format(x=matterhorntile[0],y=matterhorntile[1],z=matterhorntile[2])
HTML('<img src="' + matterhorn + '"></img>')

In [ ]:
from PIL import Image
from urllib.request import urlopen
from io import BytesIO
import numpy as np

response = urlopen(matterhorn)
buffer = BytesIO(response.read())

im = Image.open(buffer)
data = np.array(im)
im.close()

In [ ]:
print(data.shape)

In [ ]:
elevation = np.zeros((256,256,1))

In [ ]:
for y in range(data.shape[0]):
    for x in range(data.shape[1]):
        r = data[y][x][0]
        g = data[y][x][1]
        b = data[y][x][2]
        
        height = (r * 256 + g + b / 256) - 32768.
        elevation[y][x][0] = height

In [ ]:
np.min(elevation)

In [ ]:
np.max(elevation)

## Beispiel: Tiles über der Schweiz in Karte darstellen

Bounding box der Schweiz wird angenommen als:

* min_longitude: 5.95587
* max_longitude: 10.49203
* min_latitude: 45.81802
* max_latitude: 47.68038


In [ ]:
def Tile2GeoJSON(tilex, tiley, zoom):
    x0,y0,x1,y1 = TileToWGS84(tilex, tiley, zoom)
    TileGeoJSON = {
        "type": "Feature",
        "properties": {
            "name": "Tile Grid",
        },
        "geometry": {
            "type": "MultiLineString",
            "coordinates": [
                 [[x0, y0], [x0,y1]],
                 [[x0, y0], [x1,y0]],
                 [[x0, y1], [x1,y1]],
                 [[x1, y0], [x1,y1]]
              ]
            }
        }
        
    return TileGeoJSON


In [ ]:
s = Tile2GeoJSON(132,89,8)
print(s)

In [ ]:
def genTiles(minlng,minlat,maxlng,maxlat, zoom):
    t0 = WGS84ToTile(minlng,minlat,zoom)
    t1 = WGS84ToTile(maxlng,maxlat,zoom)
    
    x0 = t0[0]
    y0 = t1[1]
    x1 = t1[0]
    y1 = t0[1]
    
    alltiles = []
    
    for y in range(y0,y1+1):
        for x in range(x0,x1+1):
            alltiles.append([x,y, zoom])
    
    return alltiles    

In [ ]:
tiles = genTiles(5.95587, 45.81802, 10.49203, 47.68038, 10)
print(tiles)

In [ ]:
center = [0.5*(45.81802+47.68038), 0.5*(5.95587+10.49203)]
center

In [ ]:
import folium
from folium import GeoJson


m = folium.Map(location=center, zoom_start=5, tiles='Stamen Toner')


folium.Marker([45.81802, 5.95587], popup='BBox CH').add_to(m)
folium.Marker([47.68038, 10.49203], popup='BBox CH').add_to(m)

for tile in tiles:
    json = Tile2GeoJSON(tile[0], tile[1], tile[2])
    GeoJson(json).add_to(m)

m